In [1]:
import scipy.io
import scipy.signal as signal
from scipy.signal import butter, lfilter
import numpy as np
import pandas as pd
from mne.preprocessing import ICA
from mne.channels import make_standard_montage
%matplotlib qt
%gui qt
from time import time
from matplotlib import pyplot as plt
import mne
from mne.preprocessing import corrmap
from mne.preprocessing import find_eog_events, regress_artifact
from mne.datasets import sample
from mne.viz import plot_raw
rawww= mne.io.read_raw_eeglab('sub-001-eeg-sub-001_task-AudioCueWalkingStudy_run-02_eeg.set')
raww=rawww.set_channel_types({'TIBR1': 'emg'})
raww=rawww.set_channel_types({'TIBR2': 'emg'})
raww=rawww.set_channel_types({'TIBL1': 'emg'})
raww=rawww.set_channel_types({'TIBL2': 'emg'})
raww=rawww.set_channel_types({'HIP': 'emg'})
raww=rawww.set_channel_types({'KNEE': 'emg'})
raww=rawww.set_channel_types({'ANKLE': 'emg'})
raw=raww.pick_types( eeg=True, emg=False) 
all_events, all_event_id = mne.events_from_annotations(raw)

Reading D:\jupyter_folder\Razin_code\dfa-master\sub-001-eeg-sub-001_task-AudioCueWalkingStudy_run-02_eeg.fdt


<ipython-input-1-9f59ed4b81f0>:17: RuntimeWarning: Data file name in EEG.data (sub-001_task-AudioCueWalkingStudy_run-02_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001-eeg-sub-001_task-AudioCueWalkingStudy_run-02_eeg.fdt).
  rawww= mne.io.read_raw_eeglab('sub-001-eeg-sub-001_task-AudioCueWalkingStudy_run-02_eeg.set')


Used Annotations descriptions: ['100', '101', '103', '109', '113', '114', '119', '151', '153', '159', '200', '201', '203', '209', '251', '253', '259', '333', '444', '999']


In [2]:
my_events= mne.pick_events(all_events, include=[18, 19, 20])
epochs = mne.Epochs(raw,my_events, tmin=0.0, tmax=3.0, baseline=(0, 0))
data_3=epochs['18']
data_4=epochs['19']
data_9=epochs['20']

data_33=data_3.to_data_frame()
data_44=data_4.to_data_frame()
data_99=data_9.to_data_frame()

data_33.drop('time', axis=1, inplace=True)
data_33.drop('condition', axis=1, inplace=True)
data_33.drop('epoch', axis=1, inplace=True)
data_33.drop([0],axis=0,inplace=True)

data_44.drop('time', axis=1, inplace=True)
data_44.drop('condition', axis=1, inplace=True)
data_44.drop('epoch', axis=1, inplace=True)
data_44.drop([0],axis=0,inplace=True)

data_99.drop('time', axis=1, inplace=True)
data_99.drop('condition', axis=1, inplace=True)
data_99.drop('epoch', axis=1, inplace=True)
data_99.drop([0],axis=0,inplace=True)

data_333=data_33.to_numpy()
data_444=data_44.to_numpy()
data_999=data_99.to_numpy()

culmn_3 , row_3 =data_333.shape
culmn_4 , row_4 =data_444.shape
culmn_9, row_9=data_999.shape

eeg_333=np.empty([row_3,culmn_3])
eeg_444=np.empty([row_4,culmn_4])
eeg_999=np.empty([row_9,culmn_9])

eeg3=np.reshape(data_333,(row_3,culmn_3))
eeg4=np.reshape(data_444,(row_4,culmn_4))
eeg9=np.reshape(data_999,(row_9,culmn_9))

Not setting metadata
Not setting metadata
32 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 8 events and 1537 original time points ...
0 bad epochs dropped
Loading data for 16 events and 1537 original time points ...
0 bad epochs dropped
Loading data for 8 events and 1537 original time points ...
0 bad epochs dropped


In [11]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

filsig_eeg = butter_bandpass_filter(eeg3, 4.5, 6.5, 250, order=5) 


In [12]:
x=filsig_eeg[3]

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss

# detrended fluctuation analysis

def calc_rms(x, scale):
    """
    windowed Root Mean Square (RMS) with linear detrending.
    
    Args:
    -----
      *x* : numpy.array
        one dimensional data vector
      *scale* : int
        length of the window in which RMS will be calculaed
    Returns:
    --------
      *rms* : numpy.array
        RMS data in each window with length len(x)//scale
    """
    # making an array with data divided in windows
    shape = (x.shape[0]//scale, scale)
    X = np.lib.stride_tricks.as_strided(x,shape=shape)
    # vector of x-axis points to regression
    scale_ax = np.arange(scale)
    rms = np.zeros(X.shape[0])
    for e, xcut in enumerate(X):
        coeff = np.polyfit(scale_ax, xcut, 1)
        xfit = np.polyval(coeff, scale_ax)
        # detrending and computing RMS of each window
        rms[e] = np.sqrt(np.mean((xcut-xfit)**2))
    return rms

def dfa(x, scale_lim=[5,9], scale_dens=0.128, show=False):
    """
    Detrended Fluctuation Analysis - measures power law scaling coefficient
    of the given signal *x*.

    More details about the algorithm you can find e.g. here:
    Hardstone, R. et al. Detrended fluctuation analysis: A scale-free 
    view on neuronal oscillations, (2012).

    Args:
    -----
      *x* : numpy.array
        one dimensional data vector
      *scale_lim* = [5,9] : list of length 2 
        boundaries of the scale, where scale means windows among which RMS
        is calculated. Numbers from list are exponents of 2 to the power
        of X, eg. [5,9] is in fact [2**5, 2**9].
        You can think of it that if your signal is sampled with F_s = 128 Hz,
        then the lowest considered scale would be 2**5/128 = 32/128 = 0.128,
        so 250 ms.
      *scale_dens* = 0.25 : float
        density of scale divisions, eg. for 0.25 we get 2**[5, 5.25, 5.5, ... ] 
      *show* = False
        if True it shows matplotlib log-log plot.
    Returns:
    --------
      *scales* : numpy.array
        vector of scales (x axis)
      *fluct* : numpy.array
        fluctuation function values (y axis)
      *alpha* : float
        estimation of DFA exponent
    """
    # cumulative sum of data with substracted offset
    y = np.cumsum(x - np.mean(x))
    scales = (2**np.arange(scale_lim[0], scale_lim[1], scale_dens)).astype(np.int)
    fluct = np.zeros(len(scales))
    # computing RMS for each window
    for e, sc in enumerate(scales):
        fluct[e] = np.sqrt(np.mean(calc_rms(y, sc)**2))
    # fitting a line to rms data
    coeff = np.polyfit(np.log2(scales), np.log2(fluct), 1)
    if show:
        fluctfit = 2**np.polyval(coeff,np.log2(scales))
        plt.loglog(scales, fluct, 'bo')
        plt.loglog(scales, fluctfit, 'r', label=r'$\alpha$ = %0.2f'%coeff[0])
        plt.title('DFA')
        plt.xlabel(r'$\log_{10}$(time window)')
        plt.ylabel(r'$\log_{10}$<F(t)>')
        plt.legend()
        plt.show()
    return scales, fluct, coeff[0]


#if __name__=='__main__':
 #   n = 1000
 #   x = np.random.randn(n)
    # computing DFA of signal envelope
  #  x = np.abs(ss.hilbert(x))
  #  scales, fluct, alpha = dfa(x, show=1)
  #  print(scales)
   # print(fluct)
   # print("DFA exponent: {}".format(alpha))

if __name__=='__main__':
    n = 1000
    # computing DFA of signal envelope
    x = np.abs(ss.hilbert(x))
    scales, fluct, alpha = dfa(x, show=1)
    print(scales)
    print(fluct)
    print("DFA exponent: {}".format(alpha))


<ipython-input-13-36b915ca1b2f>:70: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scales = (2**np.arange(scale_lim[0], scale_lim[1], scale_dens)).astype(np.int)


[ 32  34  38  41  45  49  54  59  65  71  77  84  92 101 110 121 132 144
 158 172 188 206 225 246 269 294 321 351 383 419 458 500]
[ 5.94426418  6.74764748  8.21443482  9.33900792 11.20066955 13.19420511
 18.72224098 17.58456817 21.10828743 23.63294271 27.36654255 29.64463696
 33.7075092  37.89985541 41.75919668 45.02539626 47.9196058  50.91050135
 47.45996113 54.92766649 52.40203782 53.26353905 54.92041518 58.62322224
 61.64781689 69.0863891  71.2963617  71.43589123 75.68450567 77.64978659
 84.88860865 90.92616735]
DFA exponent: 0.9027541505883351
